In [1]:
from transformers import ZeroShotClassificationPipeline, AutoTokenizer, AutoConfig, T5ForSequenceClassification 
import torch

In [2]:
original_id = 'google-t5/t5-large'
config_path = r'C:\Users\lachl\PycharmProjects\bart-large-mnli-recreation\runs\google-t5\t5-large\2024-06-16--15-46-56\config_checkpoint'
checkpoint_path = r'C:\Users\lachl\PycharmProjects\bart-large-mnli-recreation\runs\google-t5\t5-large\2024-06-16--15-46-56\checkpoints\checkpoint_2'

In [3]:
config = AutoConfig.from_pretrained(config_path)

id2label = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}

config.label2id = {v:k for k,v in id2label.items()}
config.max_length = 2048

In [4]:

tokenizer = AutoTokenizer.from_pretrained(original_id)
model = T5ForSequenceClassification.from_pretrained(
    checkpoint_path,
    device_map='cuda:0',
    config=config,
)

classifier = ZeroShotClassificationPipeline(model=model, tokenizer=tokenizer)

C:\Users\lachl\PycharmProjects\bart-large-mnli-recreation\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
sequence_to_classify = "one day I will see the world."
candidate_labels = ['travel', 'cooking', 'dancing', 'pessimism', 'optimism', 'negative sentiment', 'positive sentiment', 'neutral sentiment']
result = classifier(sequence_to_classify, candidate_labels, multi_label=True)
#{'labels': ['travel', 'dancing', 'cooking'],
# 'scores': [0.9938651323318481, 0.0032737774308770895, 0.002861034357920289],
# 'sequence': 'one day I will see the world'}

result_presented = list(zip(result['labels'], [round(s,3) for s in result['scores']]))
result_presented


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[('travel', 0.955),
 ('positive sentiment', 0.856),
 ('optimism', 0.817),
 ('pessimism', 0.151),
 ('negative sentiment', 0.079),
 ('neutral sentiment', 0.079),
 ('dancing', 0.05),
 ('cooking', 0.033)]